In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import logging

module_path = os.path.abspath(os.path.join("../../.."))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from pathlib import Path

import torch
import torch.utils.data
from torchvision import transforms, datasets
import numpy as np
import matplotlib
"""matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})"""
import matplotlib.pyplot as plt

import pvi
from pvi.models import ClassificationBNNLocalRepam
from pvi.clients import Client
from pvi.servers import SequentialServer
from pvi.servers import SynchronousServer
from pvi.distributions import MeanFieldGaussianDistribution, MeanFieldGaussianFactor
from pvi.utils.training_utils import EarlyStopping

In [2]:
data_dir = Path("../../../data")
cache_dir = Path("../../../experiment_cache/fekmnist")

## Define various functions for splitting data and recording performance metrics.

In [3]:
# Data splitting functions.
def homogeneous_split(data, num_clients=100, seed=42):
    # Set numpy's random seed.
    np.random.seed(seed)
    
    perm = np.random.permutation(len(data["x"]))
    client_data = []
    for i in range(num_clients):
        client_idx = perm[i::num_clients]
        client_data.append({"x": data["x"][client_idx], "y": data["y"][client_idx]})
    
    return client_data

def heterogeneous_split(data, num_clients=100):
    # Set numpy's random seed.
    listed = np.argsort(data["y"])

    sorted_x = data["x"][listed[::1]]
    sorted_y = data["y"][listed[::1]]
    
    l = len(data["y"])
    client_size = int(np.ceil(l/num_clients))
    
    client_data = []
    
    for i in range(num_clients):        
        client_data.append({"x": sorted_x[i*client_size:(i+1)*client_size], 
                            "y": sorted_y[i*client_size:(i+1)*client_size]})
    
    return client_data


# Performance metric function. Why is the batch size 512 when the split gives 600? => Mini batches?
def performance_metrics(client, data, batch_size=512):
    dataset = torch.utils.data.TensorDataset(data["x"], data["y"])
    loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=False)
    
    device = client.config["device"]
    
    if device == "cuda":
        loader.pin_memory = True
        
    preds, mlls = [], []
    for (x_batch, y_batch) in loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        
        pp = client.model_predict(x_batch)
        preds.append(pp.component_distribution.probs.mean(1).cpu())
        mlls.append(pp.log_prob(y_batch).cpu())
        
    mll = torch.cat(mlls).mean()
    preds = torch.cat(preds)
    acc = sum(torch.argmax(preds, dim=-1) == loader.dataset.tensors[1]) / len(
        loader.dataset.tensors[1]
    )
    
    return {"mll": mll, "acc": acc}

## Data preprocessing

In [4]:
transform_train = transforms.Compose([transforms.ToTensor()])
transform_test = transforms.Compose([transforms.ToTensor()])

train_set = torch.from_numpy(np.load("../../../../data/Kuzushiji_49/k49-train-imgs.npz")['arr_0'])
train_labels = torch.from_numpy(np.load("../../../../data/Kuzushiji_49/k49-train-labels.npz")['arr_0'])
test_set = torch.from_numpy(np.load("../../../../data/Kuzushiji_49/k49-test-imgs.npz")['arr_0'])
test_labels = torch.from_numpy(np.load("../../../../data/Kuzushiji_49/k49-test-labels.npz")['arr_0'])

In [5]:
train_data = {
    "x": ((train_set - 0) / 255).reshape(-1, 28 * 28),
    "y": train_labels,
}

test_data = {
    "x": ((test_set- 0) / 255).reshape(-1, 28 * 28),
    "y": test_labels,
}
print(f"Training Data Length: ", len(train_data["x"]))
print(f"Testing Data Length: ", len(test_data["x"]))

Training Data Length:  232365
Testing Data Length:  38547


In [6]:
# Get client splits.
client_data = homogeneous_split(train_data, 25, seed=49)

## Define configuration for server and clients

In [21]:
model_config = {
    "input_dim": 784,
    "latent_dim": 200,
    "output_dim": 49,
    "num_layers": 1,
    "num_predictive_samples": 100,
    "prior_var": 1.0,
}

client_config = {
    "damping_factor": 1.0,
    "optimiser": "Adam",
    "optimiser_params": {"lr": 2e-4},
    "sigma_optimiser_params": {"lr": 2e-4},
    "early_stopping": EarlyStopping(25, score_name="elbo", stash_model=True),
    "performance_metrics": performance_metrics,
    "batch_size": 512,
    "epochs": 5000,
    "print_epochs": np.inf,
    "num_elbo_samples": 10,
    "valid_factors": False,
    "device": "cpu",
    "init_var": 1e-3,
    "verbose": True,
    "divergence": "KLD",
    "alpha": 1.0
}

server_config = {
    **client_config,
    #100
    "max_iterations": 30,
}

## Set up model etc.

In [22]:
device = server_config["device"]

model = ClassificationBNNLocalRepam(config=model_config)

# Initial parameters.
init_q_std_params = {
    "loc": torch.zeros(size=(model.num_parameters,)).to(device).uniform_(-0.1, 0.1),
    "scale": torch.ones(size=(model.num_parameters,)).to(device) 
    * client_config["init_var"] ** 0.5,
}

prior_std_params = {
    "loc": torch.zeros(size=(model.num_parameters,)).to(device),
    "scale": model_config["prior_var"] ** 0.5 
    * torch.ones(size=(model.num_parameters,)).to(device),
}

init_factor_nat_params = {
    "np1": torch.zeros(model.num_parameters).to(device),
    "np2": torch.zeros(model.num_parameters).to(device),
}

p = MeanFieldGaussianDistribution(
    std_params=prior_std_params, is_trainable=False
)
init_q = MeanFieldGaussianDistribution(
    std_params=init_q_std_params, is_trainable=False
)

clients = []
size_training = 0
for i in range(25):
    data_i = client_data[i]
    size_training += len(data_i["x"])
    t_i = MeanFieldGaussianFactor(nat_params=init_factor_nat_params)
    clients.append(
        Client(
            data=data_i,
            model=model,
            t=t_i,
            config=client_config,
            val_data=test_data
        )
    )    
server = SequentialServer(model=model, p=p, clients=clients, config=server_config, init_q=init_q, data=train_data, val_data=test_data)

## Run PGVI!

In [23]:
i = 0
while not server.should_stop():
    server.tick()

    # Obtain performance metrics.
    metrics = server.log["performance_metrics"][-1]
    print("Iterations: {}.".format(i))
    print("Time taken: {:.3f}.".format(metrics["time"]))
    print(
      "Test mll: {:.3f}. Test acc: {:.3f}.".format(
          metrics["val_mll"], metrics["val_acc"]
      )
    )
    print(
      "Train mll: {:.3f}. Train acc: {:.3f}.\n".format(
          metrics["train_mll"], metrics["train_acc"]
      )
    )
    i += 1

0it [00:00, ?it/s]

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

Iterations: 29.
Time taken: 32136.122.
Test mll: -0.887. Test acc: 0.774.
Train mll: -0.421. Train acc: 0.891.



In [9]:
model_config2 = {
    "input_dim": 784,
    "latent_dim": 200,
    "output_dim": 49,
    "num_layers": 1,
    "num_predictive_samples": 100,
    "prior_var": 1.0,
}

client_config2 = {
    "damping_factor": 1.0,
    "optimiser": "Adam",
    "optimiser_params": {"lr": 2e-4},
    "sigma_optimiser_params": {"lr": 2e-4},
    "early_stopping": EarlyStopping(25, score_name="elbo", stash_model=True),
    "performance_metrics": performance_metrics,
    "batch_size": 512,
    "epochs": 5000,
    "print_epochs": np.inf,
    "num_elbo_samples": 10,
    "valid_factors": False,
    "device": "cpu",
    "init_var": 1e-3,
    "verbose": True,
    "divergence": "AR",
    "alpha": 2.5,
}

server_config2 = {
    **client_config2,
    #100
    "max_iterations": 30,
}

In [10]:
device2 = server_config2["device"]

model2 = ClassificationBNNLocalRepam(config=model_config2)

# Initial parameters.
init_q_std_params2 = {
    "loc": torch.zeros(size=(model2.num_parameters,)).to(device2).uniform_(-0.1, 0.1),
    "scale": torch.ones(size=(model2.num_parameters,)).to(device2) 
    * client_config2["init_var"] ** 0.5,
}

prior_std_params2 = {
    "loc": torch.zeros(size=(model2.num_parameters,)).to(device2),
    "scale": model_config2["prior_var"] ** 0.5 
    * torch.ones(size=(model2.num_parameters,)).to(device2),
}

init_factor_nat_params2 = {
    "np1": torch.zeros(model2.num_parameters).to(device2),
    "np2": torch.zeros(model2.num_parameters).to(device2),
}

p2 = MeanFieldGaussianDistribution(
    std_params=prior_std_params2, is_trainable=False
)
init_q2 = MeanFieldGaussianDistribution(
    std_params=init_q_std_params2, is_trainable=False
)

clients2 = []
size_training = 0
for i in range(25):
    data_i = client_data[i]
    size_training += len(data_i["x"])
    t_i = MeanFieldGaussianFactor(nat_params=init_factor_nat_params2)
    clients2.append(
        Client(
            data=data_i,
            model=model2,
            t=t_i,
            config=client_config2,
            val_data=test_data
        )
    )    
server2 = SequentialServer(model=model2, p=p2, clients=clients2, config=server_config2, init_q=init_q2, data=train_data, val_data=test_data)


In [11]:
i = 0
while not server2.should_stop():
    server2.tick()

    # Obtain performance metrics.
    metrics2 = server2.log["performance_metrics"][-1]
    print("Iterations: {}.".format(i))
    print("Time taken: {:.3f}.".format(metrics2["time"]))
    print(
      "Test mll: {:.3f}. Test acc: {:.3f}.".format(
          metrics2["val_mll"], metrics2["val_acc"]
      )
    )
    print(
      "Train mll: {:.3f}. Train acc: {:.3f}.\n".format(
          metrics2["train_mll"], metrics2["train_acc"]
      )
    )
    i += 1

0it [00:00, ?it/s]

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

Iterations: 29.
Time taken: 36112.065.
Test mll: -0.774. Test acc: 0.810.
Train mll: -0.270. Train acc: 0.931.



In [24]:
for metric in server2.log["performance_metrics"]:
    print(metric)

{'time': 4009.795433998108, 'perf_counter': 1858.4629970001988, 'process_time': 3802.067536, 'communications': 1, 'iterations': 0, 'train_mll': tensor(-0.9060), 'train_acc': tensor(0.7973), 'val_mll': tensor(-1.5268), 'val_acc': tensor(0.6792)}
{'time': 4265.101706027985, 'perf_counter': 2113.7672795830294, 'process_time': 4298.779971, 'communications': 2, 'iterations': 0, 'train_mll': tensor(-0.8434), 'train_acc': tensor(0.7933), 'val_mll': tensor(-1.3492), 'val_acc': tensor(0.6755)}
{'time': 4591.117543935776, 'perf_counter': 2439.780580124818, 'process_time': 4925.277109999999, 'communications': 3, 'iterations': 0, 'train_mll': tensor(-0.7800), 'train_acc': tensor(0.8045), 'val_mll': tensor(-1.2870), 'val_acc': tensor(0.6913)}
{'time': 4799.267810821533, 'perf_counter': 2647.929230623413, 'process_time': 5331.278985, 'communications': 4, 'iterations': 0, 'train_mll': tensor(-0.7472), 'train_acc': tensor(0.8113), 'val_mll': tensor(-1.2536), 'val_acc': tensor(0.6977)}
{'time': 4926.81

In [14]:
model_config3 = {
    "input_dim": 784,
    "latent_dim": 200,
    "output_dim": 49,
    "num_layers": 1,
    "num_predictive_samples": 100,
    "prior_var": 1.0,
}

client_config3 = {
    "damping_factor": 1.0,
    "optimiser": "Adam",
    "optimiser_params": {"lr": 2e-4},
    "sigma_optimiser_params": {"lr": 2e-4},
    "early_stopping": EarlyStopping(25, score_name="elbo", stash_model=True),
    "performance_metrics": performance_metrics,
    "batch_size": 512,
    "epochs": 5000,
    "print_epochs": np.inf,
    "num_elbo_samples": 10,
    "valid_factors": False,
    "device": "cpu",
    "init_var": 1e-3,
    "verbose": True,
    "divergence": "KLD",
    "alpha": 1.0
}

server_config3 = {
    **client_config3,
    #100
    "max_iterations": 5,
}

In [15]:
client_data_1 = homogeneous_split(train_data,1)

In [16]:
client_data_1[0]["y"]

tensor([46, 47, 12,  ..., 24, 13,  0], dtype=torch.uint8)

In [19]:
device3 = server_config3["device"]

model3 = ClassificationBNNLocalRepam(config=model_config3)

# Initial parameters.
init_q_std_params3 = {
    "loc": torch.zeros(size=(model3.num_parameters,)).to(device3).uniform_(-0.1, 0.1),
    "scale": torch.ones(size=(model3.num_parameters,)).to(device3) 
    * client_config3["init_var"] ** 0.5,
}

prior_std_params3 = {
    "loc": torch.zeros(size=(model3.num_parameters,)).to(device3),
    "scale": model_config3["prior_var"] ** 0.5 
    * torch.ones(size=(model3.num_parameters,)).to(device3),
}

init_factor_nat_params3 = {
    "np1": torch.zeros(model3.num_parameters).to(device3),
    "np2": torch.zeros(model3.num_parameters).to(device3),
}

p3 = MeanFieldGaussianDistribution(
    std_params=prior_std_params3, is_trainable=False
)
init_q3 = MeanFieldGaussianDistribution(
    std_params=init_q_std_params3, is_trainable=False
)

clients3 = []
size_training = 0
for i in range(1):
    data_i = client_data_1[i]
    size_training += len(data_i["x"])
    t_i = MeanFieldGaussianFactor(nat_params=init_factor_nat_params3)
    clients3.append(
        Client(
            data=data_i,
            model=model3,
            t=t_i,
            config=client_config3,
            val_data=test_data
        )
    )    
server3 = SequentialServer(model=model3, p=p3, clients=clients3, config=server_config3, init_q=init_q3, data=train_data, val_data=test_data)


In [20]:
i = 0
while not server3.should_stop():
    server3.tick()

    # Obtain performance metrics.
    metrics = server3.log["performance_metrics"][-1]
    print("Iterations: {}.".format(i))
    print("Time taken: {:.3f}.".format(metrics["time"]))
    print(
      "Test mll: {:.3f}. Test acc: {:.3f}.".format(
          metrics["val_mll"], metrics["val_acc"]
      )
    )
    print(
      "Train mll: {:.3f}. Train acc: {:.3f}.\n".format(
          metrics["train_mll"], metrics["train_acc"]
      )
    )
    i += 1

0it [00:00, ?it/s]

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

Iterations: 0.
Time taken: 21882.480.
Test mll: -0.618. Test acc: 0.843.
Train mll: -0.251. Train acc: 0.942.



0it [00:00, ?it/s]

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

Iterations: 3.
Time taken: 30302.330.
Test mll: -0.613. Test acc: 0.846.
Train mll: -0.247. Train acc: 0.944.



0it [00:00, ?it/s]

Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

Iterations: 4.
Time taken: 43094.631.
Test mll: -0.614. Test acc: 0.845.
Train mll: -0.244. Train acc: 0.944.



In [26]:
train_acc = []
val_acc = []
times = []
comms = []
train_acc.append(100.0)
val_acc.append(100.0)
times.append(1.0)
comms.append(0.0)
for metrics in server.log["performance_metrics"]:
    train_acc.append(100-metrics["train_acc"]*100)
    val_acc.append(100-metrics["val_acc"]*100)
    times.append(metrics["time"]/25)
    comms.append(metrics["communications"])
    
train_acc2 = []
val_acc2 = []
times2 = []
comms2 = []
train_acc2.append(100.0)
val_acc2.append(100.0)
times2.append(1.0)
comms2.append(0.0)
for metrics in server2.log["performance_metrics"]:
    train_acc2.append(100-metrics["train_acc"]*100)
    val_acc2.append(100-metrics["val_acc"]*100)
    times2.append(metrics["time"])
    comms2.append(metrics["communications"])
    
train_acc3 = []
val_acc3 = []
times3 = []
comms3 = []
train_acc3.append(100.0)
val_acc3.append(100.0)
times3.append(1.0)
comms3.append(0.0)
for metrics in server3.log["performance_metrics"]:
    train_acc3.append(100-metrics["train_acc"]*100)
    val_acc3.append(100-metrics["val_acc"]*100)
    times3.append(metrics["time"])
    comms3.append(metrics["communications"])
    
plt.figure(figsize=(6,4))
plt.xscale("log")
#plt.yscale("log")
plt.ylim(0,40)
plt.plot(times, val_acc, linewidth=1, label=r"PVI, KL ($\alpha=1.0$)")
plt.plot(times2, val_acc2, linewidth=1, label=r"PGVI, $D_{AR}^{(2.5)}$ $\alpha=2.5$")
plt.plot(times3, val_acc3, linewidth=1, label=r"VI, KL ($\alpha=1.0$)")
plt.grid(visible=True)
#plt.xlabel("Communication Rounds")
plt.xlabel("Time")
plt.ylabel("% Testing Error")
plt.legend(loc="upper right", frameon=False)
#plt.savefig("K-49_PGVI_VI_PVI.pgf", pad_inches=0)
plt.show()

/var/folders/6x/7wzwn4g54n38_8rfbfx611n00000gn/T/ipykernel_43572/897979989.py:56: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [33]:
plt.figure(figsize=(6,4))
#plt.xscale("log")
#plt.yscale("log")
plt.ylim(10,50)
plt.plot(comms2, val_acc2, linewidth=1, label=r"PGVI, $D_{AR}^{(2.5)}$ ($\alpha=2.5$)")
plt.plot(comms, val_acc, linewidth=1, label=r"PVI, KL ($\alpha=1.0$)")

#plt.plot(times3, val_acc3, linewidth=1, label=r"VI, KL ($\alpha=1.0$)")
plt.grid(visible=True)
plt.xlabel("Communication Rounds")
#plt.xlabel("Time")
plt.ylabel("% Testing Error")
plt.legend(loc="upper right", frameon=False)
#plt.savefig("K-49_PGVI_PVI.pgf", pad_inches=0)
plt.show()

/var/folders/6x/7wzwn4g54n38_8rfbfx611n00000gn/T/ipykernel_43572/52514647.py:15: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
